In [1]:
import pandas as pd
import numpy as np
import re           ## regular expressions
import requests     ## for getting URLs
from bs4 import BeautifulSoup  ## 


Go to the border crossing web site; grab the current version of the web page; and use `BeautifulSoup` to dissect it.

In [7]:
url = "http://www.cbsa-asfc.gc.ca/bwt-taf/menu-eng.html#_s1"
response = requests.get(url)                ## get full web page
soup = BeautifulSoup(response.text,"lxml")  ## parse the response

The `.findChildren()` method returns a list of elements of a particular type within an object

In [9]:
tab=soup.findChildren("table")[0]  ## first table on the page   
rows = tab.findChildren("tr")      ## list of all rows in the table
rows[0]                            ## first row is the header

<tr class="bg-primary">
<th>CBSA Office</th>
<th>Commercial Flow</th>
<th>Travellers Flow</th>
<th>Updated</th>
</tr>

In [4]:
rows[1]   ## second row is data

<tr><th><b>St. Stephen</b><br/>St. Stephen, NB/Calais, ME<br/></th><td>Not applicable</td><td>No delay</td><td><time datetime="2016-04-04T23:14-0300">2016-04-04 23:14 ADT</time></td></tr>

Now we have to figure out how to extract the information from this. We can use

- `.findChildren("th")` to get the first (`<th>...</th>`) element
- `.findChildren("td")` to get subsequent (`<td>...</td>`) elements

In [10]:
ff = rows[1].findChildren("th")
print(ff)

[<th><b>St. Stephen</b><br/>St. Stephen, NB/Calais, ME<br/></th>]


we'd like to use `.get_text()` to simplify this, but it's a little *too* effective ... it drops the `<br/>` tag that's used to make a new line in the table ...

In [6]:
ff[0].get_text()

'St. StephenSt. Stephen, NB/Calais, ME'

Here's one way to do it: use `re.split()` to split a text version of the object on its HTML tags

In [7]:
rm = np.array(re.split("<[a-z/]+>",format(ff[0])))
print(rm)

['' '' 'St. Stephen' '' 'St. Stephen, NB/Calais, ME' '' '']


This will work as long as the format is consistent and we always want to get elements 2 and 4 of the result ...

In [8]:
(loc1,loc2) = tuple(rm[[2,4]])
print((loc1,loc2))

('St. Stephen', 'St. Stephen, NB/Calais, ME')


What about the rest of the data in the row?

In [11]:
ff2 = rows[1].findChildren("td")
print(ff2)

[<td>Not applicable</td>, <td>No delay</td>, <td><time datetime="2016-04-05T12:46-0300">2016-04-05 12:46 ADT</time></td>]


The delay information needs to be translated as follows:
    
- "Not applicable" = `NaN`, which we can do more idiomatically by setting the value to `None`
- "No delay" = 0 
- "[xx] minutes" = floating-point value of xx

In [12]:
def delay_transform(x):
    '''transform raw delay information to a numeric value'''
    if x=="Not applicable":
            p = None
    elif x=="No delay":
            p = 0
    elif "minute" in x:
            num = (x.split(' '))[0]
            p = float(num)
    else:
            raise ValueError("can't convert: "+x)
    return(p)


In [13]:
ff2[0]

<td>Not applicable</td>

In [14]:
ff2[0].get_text()

'Not applicable'

In [11]:
print((delay_transform(ff2[0].get_text()),
      delay_transform(ff2[1].get_text())))

(None, 0)


In [12]:
ff2[2].get_text()

'2016-04-04 23:14 ADT'

Now process the entire table ...

In [13]:
res = []
for r in rows[1:]:
    res.append([])   ## add a new row/empty list to the results
    ff = r.findChildren("th")[0]
    rm = np.array(re.split("<[a-z/]+>",format(ff)))
    res[-1].append(rm[2])  ## add first loc
    res[-1].append(rm[4])  ## add second loc
    ff2 = r.findChildren("td")
    res[-1].append(delay_transform(ff2[0].get_text()))  ## commercial
    res[-1].append(delay_transform(ff2[1].get_text()))  ## traveler
    res[-1].append(pd.to_datetime(ff2[2].get_text()))   ## time
p = pd.DataFrame(res)
p.columns = ["loc1","loc2","delay_commercial","delay_traveler","time"]
p.index = p.time
p.drop("time",axis=1,inplace=True)
p

,loc1,loc2,delay_commercial,delay_traveler
time,,,,
2016-04-04 23:14:00,St. Stephen,"St. Stephen, NB/Calais, ME",NaN,0.0
2016-04-04 23:14:00,St. Stephen 3rd Bridge,"St. Stephen, NB/Calais, ME",0.0,0.0
2016-04-04 23:03:00,Edmundston,"Edmundston, NB/Madawaska, ME",0.0,0.0
2016-04-04 23:14:00,Woodstock Road,"Belleville, NB/Houlton, ME",0.0,0.0
2016-04-05 02:02:00,Stanstead,"Stanstead, QC/Derby Line, VT",0.0,0.0
2016-04-05 02:14:00,St-Armand/Philipsburg,"St. Armand, QC/Highgate Springs, VT",0.0,0.0
2016-04-04 23:13:00,St-Bernard-de-Lacolle,"Lacolle, QC/Champlain, NY",0.0,30.0
2016-04-05 02:14:00,Cornwall,"Cornwall, ON/Rooseveltown, NY",0.0,0.0
2016-04-05 02:14:00,Prescott,"Prescott, ON/Ogdensburg, NY",0.0,0.0
